In [1]:
from PIL import Image
from tqdm.auto import tqdm
from numpy import asarray
import math
import torch
from utils import utils, verification
import os
import lpips
from torchvision import transforms
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
import pyiqa

In [2]:
def check(target_FRS='t1', source_FRS='AWS', dataset='LFW', device='cuda:0', target_bin=False):

    # Valid source_FRS (string): t1, t2, t3, t4, t5, AWS, KAIROS, FACEPP
    # Valid target_FRS (string): t1, t2, t3, t4, t5
    # Valid dataset    (string): LFW, AGE, CFP
    
    print("Source FRS : "+source_FRS)
    print("Target FRS : "+target_FRS)
    print("Target Dataset : "+dataset)
    
    # You can download png files including target images and corresponding reconstructed image using Zenodo link in README file.
    # There are small ASR gap in terms of input image files (bin file or png file).
    
    device = torch.device(device)
    blackbox = utils.target_FRS(target_FRS=target_FRS, device=device)
    img_size = (112,112)
    
    if dataset=='LFW':
        target_number = 3000
        if target_FRS=='t1':
            thx=72.54239687627792
        elif target_FRS=='t2':
            thx=77.29096700560457
        elif target_FRS=='t3':
            thx=74.33573314862649
        elif target_FRS=='t4':
            thx=75.81816627143655
        elif target_FRS=='t5':
            thx=63.57666123410324
            img_size = (160,160)
            
        if target_bin:
            print("Using bin file instead of png image")
            dataset_dir = "./utils/dataset/lfw.bin"
            target_dataset = verification.load_bin(dataset_dir, img_size)
            target_ind = torch.where(torch.tensor(target_dataset[1]))[0]
            
    elif dataset=="AGE":
        target_number = 3000
        if target_FRS=='t1':
            thx=76.40837722605214
        elif target_FRS=='t2':
            thx=80.50276553148295
        elif target_FRS=='t3':
            thx=79.04721580110888
        elif target_FRS=='t4':
            thx=78.75527640762357
        elif target_FRS=='t5':
            thx=70.42746205557108
            img_size = (160,160)
            
        if target_bin:
            print("Using bin file instead of png image")
            dataset_dir = "./utils/dataset/agedb_30.bin"
            target_dataset = verification.load_bin(dataset_dir, img_size)
            target_ind = torch.where(torch.tensor(target_dataset[1]))[0]
    
    elif dataset=="CFP":
        target_number = 3500
        if target_FRS=='t1':
            thx=77.29096700560457
        elif target_FRS=='t2':
            thx=81.08320374700904
        elif target_FRS=='t3':
            thx=79.63024019452259
        elif target_FRS=='t4':
            thx=81.08320374700904
        elif target_FRS=='t5':
            thx=73.73979529168804
            img_size = (160,160)
            
        if target_bin:
            print("Using bin file instead of png image")
            dataset_dir = "./utils/dataset/cfp_fp.bin"
            target_dataset = verification.load_bin(dataset_dir, img_size)
            target_ind = torch.where(torch.tensor(target_dataset[1]))[0]
    
    result = torch.zeros(2,target_number)

    for i in tqdm(range(target_number)):
        if target_bin:
            if target_FRS == 't2':
                data_Type1 = (target_dataset[0][0][0::2][int(target_ind[i])]).unsqueeze(0).to(device)
                data_Type2 = (target_dataset[0][0][1::2][int(target_ind[i])]).unsqueeze(0).to(device)
            else:
                data_Type1 = ((target_dataset[0][0][0::2][int(target_ind[i])]-127.5)/255).unsqueeze(0).to(device)
                data_Type2 = ((target_dataset[0][0][1::2][int(target_ind[i])]-127.5)/255).unsqueeze(0).to(device)
        else:
            img_Type1 = Image.open("./recon/"+dataset+"/Type1_"+dataset+"/"+str(i)+".png")
            img_Type1 = img_Type1.resize(img_size)
            img_Type1 = img_Type1.convert('RGB')
            data_Type1 = asarray(img_Type1)
            if target_FRS == 't2':
                data_Type1 = ((torch.Tensor(data_Type1))).permute(2,0,1).unsqueeze(0).to(device)
            else:
                data_Type1 = ((torch.tensor(data_Type1)-127.5)/255).permute(2,0,1).unsqueeze(0).to(device)

            img_Type2 = Image.open("./recon/"+dataset+"/Type2_"+dataset+"/"+str(i)+".png")
            img_Type2 = img_Type2.resize(img_size)
            img_Type2 = img_Type2.convert('RGB')
            data_Type2 = asarray(img_Type2)
            if target_FRS == 't2':
                data_Type2 = ((torch.Tensor(data_Type2))).permute(2,0,1).unsqueeze(0).to(device)
            else:
                data_Type2 = ((torch.tensor(data_Type2)-127.5)/255).permute(2,0,1).unsqueeze(0).to(device)


        img_recon = Image.open("./recon/"+dataset+"/"+source_FRS+"_"+dataset+"/"+str(i)+".png")
        img_recon = img_recon.resize(img_size)
        img_recon = img_recon.convert('RGB')
        data_recon = asarray(img_recon)
        if target_FRS == 't2':
            data_recon = ((torch.Tensor(data_recon))).permute(2,0,1).unsqueeze(0).to(device)
        else:
            data_recon = ((torch.tensor(data_recon)-127.5)/255).permute(2,0,1).unsqueeze(0).to(device)


        with torch.no_grad():
            feat_Type1 = blackbox(data_Type1)
            feat_Type2 = blackbox(data_Type2)
            feat_recon = blackbox(data_recon)

            feat_Type1 = feat_Type1/feat_Type1.norm()
            feat_Type2 = feat_Type2/feat_Type2.norm()
            feat_recon = feat_recon/feat_recon.norm()

            result[0,i]=(torch.acos((feat_Type1*feat_recon).sum())*180/math.pi).to("cpu")
            result[1,i]=(torch.acos((feat_Type2*feat_recon).sum())*180/math.pi).to("cpu")

    print("Type-1 ASR : "+str(round(float(100*(result[0,:] <= thx).sum(0)/target_number),2))+"%")
    print("Type-2 ASR : "+str(round(float(100*(result[1,:] <= thx).sum(0)/target_number),2))+"%")

    return result

In [3]:
# Table 8 Direct attacks
def check_direct(target='t1',target_bin=False):
    tab_lfw = check(target,target,'LFW','cuda:0',target_bin)
    tab_age = check(target,target,'AGE','cuda:0',target_bin)
    tab_cfp = check(target,target,'CFP','cuda:0',target_bin)
    return [tab_lfw,tab_age,tab_cfp]

In [4]:
# Table 8
dir_t1 = check_direct('t1')
dir_t2 = check_direct('t2')
dir_t3 = check_direct('t3')
dir_t4 = check_direct('t4')
dir_t5 = check_direct('t5')

Source FRS : t1
Target FRS : t1
Target Dataset : LFW


  0%|          | 0/3000 [00:00<?, ?it/s]

Type-1 ASR : 96.17%
Type-2 ASR : 47.87%
Source FRS : t1
Target FRS : t1
Target Dataset : AGE


  0%|          | 0/3000 [00:00<?, ?it/s]

Type-1 ASR : 99.6%
Type-2 ASR : 46.37%
Source FRS : t1
Target FRS : t1
Target Dataset : CFP


...

Type-1 ASR : 80.57%
Type-2 ASR : 55.83%
Source FRS : t5
Target FRS : t5
Target Dataset : AGE


  0%|          | 0/3000 [00:00<?, ?it/s]

Type-1 ASR : 94.47%
Type-2 ASR : 61.8%
Source FRS : t5
Target FRS : t5
Target Dataset : CFP


  0%|          | 0/3500 [00:00<?, ?it/s]

Type-1 ASR : 97.54%
Type-2 ASR : 77.71%


In [5]:
# Table 10 and 11 transfer attacks (against non-commercial targets)
# Note that paper only reports for LFW, but we can run for all 3 datasets
sources = ['t1','t2','t3','t4','t5','AWS','FACEPP','KAIROS']
def check_transfer(target='t1',dataset='LFW',target_bin=False):
    return [check(target,source,dataset,'cuda:0',target_bin) for source in sources if source !=target]

In [6]:
# Table 10 (png)
trans_t1_nb = check_transfer('t1',target_bin=False)
trans_t2_nb = check_transfer('t2',target_bin=False)
trans_t3_nb = check_transfer('t3',target_bin=False)
trans_t4_nb = check_transfer('t4',target_bin=False)
trans_t5_nb = check_transfer('t5',target_bin=False)

Source FRS : t2
Target FRS : t1
Target Dataset : LFW


  0%|          | 0/3000 [00:00<?, ?it/s]

Type-1 ASR : 61.6%
Type-2 ASR : 18.6%
Source FRS : t3
Target FRS : t1
Target Dataset : LFW


  0%|          | 0/3000 [00:00<?, ?it/s]

Type-1 ASR : 89.13%
Type-2 ASR : 36.7%
Source FRS : t4
Target FRS : t1
Target Dataset : LFW


...

Type-1 ASR : 50.57%
Type-2 ASR : 41.9%
Source FRS : FACEPP
Target FRS : t5
Target Dataset : LFW


  0%|          | 0/3000 [00:00<?, ?it/s]

Type-1 ASR : 58.37%
Type-2 ASR : 47.9%
Source FRS : KAIROS
Target FRS : t5
Target Dataset : LFW


  0%|          | 0/3000 [00:00<?, ?it/s]

Type-1 ASR : 70.53%
Type-2 ASR : 58.3%


In [7]:
# Table 12 previous attacks (against non-commercial targets)
sources = ['gauss_t1','eigen_t1','style_t1','t1']
def check_transferprev(target='t1',dataset='LFW',target_bin=False):
    return [check(target,source,dataset,'cuda:0',target_bin) for source in sources]

In [8]:
# Table 12 (png)
transp_t2_nb = check_transferprev('t2',target_bin=False)
transp_t3_nb = check_transferprev('t3',target_bin=False)
transp_t4_nb = check_transferprev('t4',target_bin=False)
transp_t5_nb = check_transferprev('t5',target_bin=False)

Source FRS : gauss_t1
Target FRS : t2
Target Dataset : LFW
self.device_id 0


  0%|          | 0/3000 [00:00<?, ?it/s]

Type-1 ASR : 1.4%
Type-2 ASR : 0.43%
Source FRS : eigen_t1
Target FRS : t2
Target Dataset : LFW
self.device_id 0


  0%|          | 0/3000 [00:00<?, ?it/s]

Type-1 ASR : 4.67%
Type-2 ASR : 1.37%
Source FRS : style_t1
Target FRS : t2
Target Dataset : LFW
self.device_id 0


...

Type-1 ASR : 5.93%
Type-2 ASR : 4.23%
Source FRS : style_t1
Target FRS : t5
Target Dataset : LFW


  0%|          | 0/3000 [00:00<?, ?it/s]

Type-1 ASR : 16.9%
Type-2 ASR : 13.6%
Source FRS : t1
Target FRS : t5
Target Dataset : LFW


  0%|          | 0/3000 [00:00<?, ?it/s]

Type-1 ASR : 55.6%
Type-2 ASR : 46.43%


In [9]:
# Table 13 comparison of image similarities on LFW dataset
device = torch.device("cpu")
iqa_metric = pyiqa.create_metric("dists",device=device)
loss_fn_alex = lpips.LPIPS(net='alex')
loss_fn_vgg = lpips.LPIPS(net='vgg')

@torch.no_grad()
def check_metric(source='t1'):
    
    result=torch.zeros(5,3000)

    for i in tqdm(range(3000)):
        img_recon = Image.open("./recon/LFW/"+source+"_LFW/"+str(i)+".png")
        img_recon = img_recon.resize((112,112))
        img_recon = img_recon.convert('RGB')
        data_recon = asarray(img_recon)
        recon = ((torch.tensor(data_recon)-127.5)/255).permute(2,0,1).unsqueeze(0)#.to(device)

        img_target = Image.open("./recon/LFW/Type1_LFW/"+str(i)+".png")
        img_target = img_target.resize((112,112))
        img_target = img_target.convert('RGB')
        data_target = asarray(img_target)
        target = ((torch.tensor(data_target)-127.5)/255).permute(2,0,1).unsqueeze(0)#.to(device)

        d_alex = loss_fn_alex(recon, target)
        d_vgg = loss_fn_vgg(recon, target)
        d_ssim = ssim((recon+1)/2,(target+1)/2 , data_range=1 , size_average=False)
        d_L2 = (recon-target).norm()

        recon = torch.FloatTensor(data_recon/255).permute(2,0,1).unsqueeze(0)
        target = torch.FloatTensor(data_target/255).permute(2,0,1).unsqueeze(0)
        d_dists = iqa_metric(recon,target)

        result[0,i]=d_alex
        result[1,i]=d_vgg
        result[2,i]=d_dists
        result[3,i]=d_ssim
        result[4,i]=d_L2
    
    if source == "gauss_t1" or source == "eigen_t1" or source == "style_t1":
        print("Source : "+source+" (previous work)")
    else:
        print("Source : "+source+" (Ours)")
    print("LPIPS (Alex) :"+str(round(float(result[0].mean()),3)))
    print("LPIPS (VGG) :"+str(round(float(result[1].mean()),3)))
    print("DISTS :"+str(round(float(result[2].mean()),3)))
    print("SSIM :"+str(round(float(result[3].mean()),3)))
    print("L2-norm :"+str(round(float(result[4].mean()),3)))
    print("=============================================")
    
    return result

Loading pretrained model DISTS from /home/cryptology96/.cache/torch/hub/checkpoints/DISTS_weights-f5e65c96.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/cryptology96/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cryptology96/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/cryptology96/anaconda3/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/home/cryptology96/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/cryptology96/anaconda3/lib/python3.9/site-packages/lpips/weights/v0.1/vgg.pth


In [10]:
similarity_gauss = check_metric('gauss_t1')
similarity_eigen = check_metric('eigen_t1')
similarity_style = check_metric('style_t1')
similarity_t1 = check_metric('t1')
similarity_aws = check_metric('AWS')

  0%|          | 0/3000 [00:00<?, ?it/s]

Source : gauss_t1 (previous work)
LPIPS (Alex) :0.531
LPIPS (VGG) :0.475
DISTS :0.394
SSIM :0.589
L2-norm :72.38


  0%|          | 0/3000 [00:00<?, ?it/s]

Source : eigen_t1 (previous work)
LPIPS (Alex) :0.338
LPIPS (VGG) :0.391
DISTS :0.285
SSIM :0.721
L2-norm :39.57


  0%|          | 0/3000 [00:00<?, ?it/s]

Source : style_t1 (previous work)
LPIPS (Alex) :0.407
LPIPS (VGG) :0.482
DISTS :0.316
SSIM :0.522
L2-norm :49.788


  0%|          | 0/3000 [00:00<?, ?it/s]

Source : t1 (Ours)
LPIPS (Alex) :0.228
LPIPS (VGG) :0.322
DISTS :0.243
SSIM :0.781
L2-norm :30.793


  0%|          | 0/3000 [00:00<?, ?it/s]

Source : AWS (Ours)
LPIPS (Alex) :0.228
LPIPS (VGG) :0.323
DISTS :0.24
SSIM :0.787
L2-norm :29.322
